In [1]:
import json
import base64
from PIL import Image
from io import BytesIO
import sagemaker
from sagemaker import get_execution_role
from sagemaker.model import Model
from sagemaker.predictor import Predictor

sagemaker.config INFO - Fetched defaults config from location: /etc/xdg/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /home/sagemaker-user/.config/sagemaker/config.yaml
sagemaker.config INFO - Applied value from config key = SageMaker.PythonSDK.Modules.Session.DefaultS3Bucket
sagemaker.config INFO - Applied value from config key = SageMaker.PythonSDK.Modules.Session.DefaultS3ObjectKeyPrefix


In [2]:
#Set variables
session = sagemaker.Session()
region = session.boto_region_name
account_id = session.account_id()

print("Region:", region)
print("Account:", account_id)

sagemaker.config INFO - Applied value from config key = SageMaker.PythonSDK.Modules.Session.DefaultS3Bucket
sagemaker.config INFO - Applied value from config key = SageMaker.PythonSDK.Modules.Session.DefaultS3ObjectKeyPrefix
Region: us-east-1
Account: 610912512102


In [3]:
ecr_image_uri = f"{account_id}.dkr.ecr.{region}.amazonaws.com/nerf-sagemaker:latest"

model_data_s3 = "s3://amazon-sagemaker-610912512102-us-east-1-0379069f3fe5/dzd_45z6d3gc7k13dz/43l9xaxd7j28pz/shared/data/nerfstudio/processed_forabot_BN_final-large2_2025-06-11_122746.tar.gz"

role = get_execution_role()
print("Execution Role:", role)

sagemaker.config INFO - Applied value from config key = SageMaker.PythonSDK.Modules.Session.DefaultS3Bucket
sagemaker.config INFO - Applied value from config key = SageMaker.PythonSDK.Modules.Session.DefaultS3ObjectKeyPrefix
Execution Role: arn:aws:iam::610912512102:role/datazone_usr_role_43l9xaxd7j28pz_504bhe16x4mjzb


In [ ]:
#Create a Sagemaker Model object
#Tells Sagemaker to 
#Use the Docker image from ECR,
#load model.tar.gz from S3,
#Use the execution role

In [4]:
model_name = "nerf-sagemaker-model"

nerf_model = Model(
    image_uri=ecr_image_uri,
    model_data=model_data_s3,
    role=role,
    name=model_name,
    sagemaker_session=session,
)

sagemaker.config INFO - Applied value from config key = SageMaker.Model.VpcConfig


In [ ]:
#Create an EndpointConfig
#Define
#The type of instance (GPU required)
#The model to use
#How much traffic to send (100%)

In [ ]:
endpoint_config_name = "nerf-sagemaker-endpoint-config"

endpoint_config = session.create_endpoint_config(
    name=endpoint_config_name,
    model_name=model_name,
    initial_instance_count=1,
    instance_type="ml.g4dn.xlarge"
)

In [ ]:
#Deploy a real time Endpoint
#Progress can be monitored at SageMaker Console → Inference → Endpoints → nerf-sagemaker-endpoint

In [ ]:
endpoint_name = "nerf-sagemaker-endpoint"

session.create_endpoint(
    endpoint_name=endpoint_name,
    config_name=endpoint_config_name
)

In [ ]:
#test the endpoint


predictor = Predictor(
    endpoint_name=endpoint_name,
    sagemaker_session=session
)

payload = json.dumps({"camera_index": 0})
resp = predictor.predict(
    payload,
    initial_args={
        "ContentType": "application/json",
        "Accept": "application/json"
    }
)

print(resp)

In [ ]:
#Decode output to display rendered NeRF frame from Sagemaker endpoint

result = json.loads(resp.decode("utf-8"))
png_bytes = base64.b64decode(result["image_base64"])
img = Image.open(BytesIO(png_bytes))
img